In [14]:
import pandas as pd
import os


# Paths
MODEL_PATH = '../models/stock_price_predictor_25061714-17.keras'
SCALER_X_PATH = '../models/scaler_X.pkl'
SCALER_Y_PATH = '../models/scaler_y.pkl'
PREDICTIONS_DIR = '../data/predictions'

# Load and preprocess data
df = pd.read_csv(os.path.join(PREDICTIONS_DIR, 'X_prediction.csv'), index_col="Date", parse_dates=True)

# Map sentiment strings to numerical values
sentiment_map = {'Bearish': -1, 'Neutral': 0, 'Bullish': 1}
df['SentimentNum'] = df['Sentiment'].map(sentiment_map)


# New column for date with Epoch time in seconds
df['Date'] = df.index.astype('int64') / 10**9



# # Select features from Processed_data.csv
X = df[["Date", "GDP growth rate (%)", "Unemployment rate (%)", "Real interest rate (%)", "Inflation rate (%)", "Population growth (%)", "Export growth (%)", "Import growth (%)", "SentimentNum"]]

X.head()


,Date,GDP growth rate (%),Unemployment rate (%),Real interest rate (%),Inflation rate (%),Population growth (%),Export growth (%),Import growth (%),SentimentNum
Date,,,,,,,,,
2025-09-01,1.756685e+09,3.2,4.5,2.2,4.2,1,1.15,1.2,1
2025-09-01,1.756685e+09,3.2,4.5,2.2,4.2,1,1.15,1.2,0
2025-09-01,1.756685e+09,3.2,4.5,2.2,4.2,1,1.15,1.2,1


In [ ]:
import joblib
from tensorflow import keras

# Load and apply the scaler
scaler_X = joblib.load(SCALER_X_PATH)
scaler_y = joblib.load(SCALER_Y_PATH)

X_scaled = scaler_X.transform(X)

# Load model and make predictions

def masked_mse(y_true, y_pred):
    mask = ~tf.math.is_nan(y_true)
    y_true = tf.where(mask, y_true, 0.0)
    y_pred = tf.where(mask, y_pred, 0.0)
    mse = tf.reduce_sum(tf.square(y_true - y_pred) * tf.cast(mask,
                        tf.float32)) / tf.reduce_sum(tf.cast(mask, tf.float32))
    return mse

model = keras.models.load_model(MODEL_PATH, custom_objects={'masked_mse': masked_mse})

predictions = model.predict(X_scaled)

# Inverse transform predictions
predictions = scaler_y.inverse_transform(predictions)

# Save predictions
pred_df = pd.DataFrame(predictions, index=df.index, columns=['AAPL', 'BTC-USD', 'GOOGL', 'MSFT'])
os.makedirs(os.path.dirname(PREDICTIONS_DIR), exist_ok=True)
pred_df.to_csv(os.path.join(PREDICTIONS_DIR, 'predictions.csv'))

print(f"Predictions saved to {os.path.join(PREDICTIONS_DIR, 'predictions.csv')}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step
Predictions saved to ../data/predictions/predictions.csv
